<a href="https://colab.research.google.com/github/AUT-Student/NLP-HW1/blob/main/NLP_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><b>In the name of God</b></center>

<b>Course</b>: Natural Language Processing
<br>
<b>Description:</b> HomeWork 1
<br>
<b>Developer</b>: Alireza Mazochi (400131075)

# Library

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
import numpy as np
import random
import math

# Dataset

In [ ]:
!gdown --id 16C0_9i0io43VfABV3-uukUjJYlM6k-2U
!unzip /content/HW1-datasets.zip

Downloading...
From: https://drive.google.com/uc?id=16C0_9i0io43VfABV3-uukUjJYlM6k-2U
To: /content/HW1-datasets.zip
100% 3.14M/3.14M [00:00<00:00, 51.9MB/s]
Archive:  /content/HW1-datasets.zip
   creating: content/HW1-datasets/
  inflating: content/HW1-datasets/train.txt  
  inflating: content/HW1-datasets/valid.txt  
  inflating: content/HW1-datasets/test_incomplete.txt  
  inflating: content/HW1-datasets/test.txt  
  inflating: content/HW1-datasets/test_incomplete_gold.txt  


# Statistical Language Model

## Counting

In [ ]:
file = open("/content/content/HW1-datasets/train.txt")

word_counter = {}
pair_counter = {}
vocabulary = set()
N = 0

for line in file.readlines():
    words = line.split()
    N += len(words)
    
    for i, word in enumerate(words):
        vocabulary.add(word)
        if word_counter.get(word) is None:
            word_counter[word] = 0
        word_counter[word] += 1
    
        if i>0:
            pair = (words[i-1], word)
            if pair_counter.get(pair) is None:
                pair_counter[pair] = 0
            pair_counter[pair] += 1

## Functions

In [ ]:
def create_unigram(delta):
  def unigram(word):
    if word in vocabulary:
      return word_counter[word] / N 
    else:
      return delta / N

  return unigram

In [ ]:
def create_bigram(delta, unigram):
    def calculate_B(delta):
        B_word = {}

        for word in vocabulary:
          B_word[word] = 0

        for pair in pair_counter:
            word1 = pair[0]
            B_word[word1] += 1
        
        for word in vocabulary:
          if B_word[word]==0:
            B_word[word] = 1

        return B_word
    
    B_word = calculate_B(delta)
    
    def bigram(word1, word2):
      p_bg = unigram(word2)
      if word1 not in vocabulary:
        return p_bg
      else:
        alpha = delta / word_counter[word1] * B_word[word1]

        return (max(pair_counter.get((word1, word2), 0) - delta, 0) / word_counter[word1]) + alpha * p_bg

    return bigram

In [ ]:
def create_statistical_dataset(path, number_words, line_limit=None, data_number_limit=None):
  file = open(path)

  X_dataset = []
  Y_dataset = []

  for i, line in enumerate(file.readlines()):
    if line_limit is not None and i >= line_limit:
      break

    all_words = line.split()

    for j in range(number_words, len(all_words)):
      words = []
      for k in range(number_words):
        word = all_words[j-k-1] 
        words.append(word)
      
      X_dataset.append(words)

      word_target = all_words[j]
      Y_dataset.append(word_target)

  print(f"Maximum Data = {len(X_dataset)}")

  if data_number_limit is not None:
    random.seed(0)
    X_dataset, Y_dataset = zip(*random.sample(list(zip(X_dataset, Y_dataset)), data_number_limit))

  X_dataset = np.array(X_dataset)
  Y_dataset = np.array(Y_dataset)
  return X_dataset, Y_dataset

## Perplexity

In [ ]:
def unigram_perplexity(path):
  file = open(path)

  counter = 0
  probability = 0

  for line in file.readlines():
    words = line.split()
    
    for word in words:
      probability += math.log(unigram(word))
      counter += 1

  log_perplexity = -1/counter*probability
  
  perplexity = math.exp(log_perplexity)

  return perplexity

In [ ]:
def bigram_perplexity(path):
  file = open(path)

  counter = 0
  probability = 0

  for line in file.readlines():
    words = line.split()
  
    for i in range(1, len(words)):
      probability += math.log(bigram(words[i-1], words[i]))
      counter += 1

  log_perplexity = -1/counter*probability  

  perplexity = math.exp(log_perplexity)

  return perplexity

## Fine-Tuning

In [ ]:
path = "/content/content/HW1-datasets/valid.txt"

best_unigram_perplexity = np.inf
best_unigram_delta = None

for delta in np.arange(0.2, 0.98, 0.02):
  unigram = create_unigram(delta=delta)

  new_unigram_perplexity = unigram_perplexity(path)

  if new_unigram_perplexity < best_unigram_perplexity:
    best_unigram_perplexity = new_unigram_perplexity
    best_unigram_delta = delta

print(f"Delta = {best_unigram_delta} Perplexity = {best_unigram_perplexity}")

Delta = 0.9599999999999995 Perplexity = 1828.493440423088


In [ ]:
path = "/content/content/HW1-datasets/valid.txt"

best_bigram_perplexity = np.inf
best_bigram_delta = None

for delta in np.arange(0.2, 0.98, 0.02):
  unigram = create_unigram(delta=0.96)
  bigram = create_bigram(delta=delta, unigram=unigram)

  new_bigram_perplexity = bigram_perplexity(path)

  if new_bigram_perplexity < best_bigram_perplexity:
    best_bigram_perplexity = new_bigram_perplexity
    best_bigram_delta = delta

print(f"Delta = {best_bigram_delta} Perplexity = {best_bigram_perplexity}")

Delta = 0.8799999999999997 Perplexity = 1318.4189504258493


## Unigram

In [ ]:
unigram = create_unigram(delta=0.96)

### Perplexity

In [ ]:
for text in ["train", "test", "valid"]:
  path = f"/content/content/HW1-datasets/{text}.txt"
  print(f"Unigram perplexity of {text} = {unigram_perplexity(path)}")

Unigram perplexity of train = 1768.173187352548
Unigram perplexity of test = 1810.0517762463387
Unigram perplexity of valid = 1828.493440423088


### Quantitative Test

In [ ]:
for dataset in ["train", "test", "valid"]:
  X_test, Y_test = create_statistical_dataset(path=f"/content/content/HW1-datasets/{dataset}.txt",
                                              number_words=0,
                                              data_number_limit=50*1000)

  best_word = None
  best_probability = -math.inf
  for condidate_word in vocabulary:
      if unigram(condidate_word) > best_probability:
          best_probability = unigram(condidate_word)
          best_word = condidate_word

  Y_test_predict = []
  for _ in range(len(Y_test)):
    Y_test_predict.append(best_word)

  print(f"{dataset} Accuracy = {accuracy_score(Y_test, Y_test_predict)}")

Maximum Data = 1063985
train Accuracy = 0.04028
Maximum Data = 85324
test Accuracy = 0.03968
Maximum Data = 132968
valid Accuracy = 0.03992


### Qualitative Test

In [ ]:
best_word = None
best_probability = -math.inf
for condidate_word in vocabulary:
    if unigram(condidate_word) > best_probability:
        best_probability = unigram(condidate_word)
        best_word = condidate_word

file = open("/content/content/HW1-datasets/test_incomplete.txt")
incomplete_lines = file.readlines()

file = open("/content/content/HW1-datasets/test_incomplete_gold.txt")
incomplete_gold_lines = file.readlines()

for i in range(len(incomplete_lines)):
    line = incomplete_lines[i]
    digit = int(line[0])
    line = line[4:]
    print(line.strip(), end=" ")
    for _ in range(digit):
        print(f"\x1b[32m{best_word}\x1b[0m", end= " ")
    print()
    
    print(incomplete_gold_lines[i])    

این سخن حقست اگر نزد سخن گستر و 
این سخن حقست اگر نزد سخن گستر برند

آنکه با یوسف صدیق چنین خواهد و 
آنکه با یوسف صدیق چنین خواهد کرد

هیچ دانی چکند صحبت او با و 
هیچ دانی چکند صحبت او با دگران

سرمه دهی بصر بری سخت خوش است و 
سرمه دهی بصر بری سخت خوش است تاجری

آتش ابراهیم را و و 
آتش ابراهیم را نبود زیان

من که اندر سر و و 
من که اندر سر جنونی داشتم

هر شیر شرزه را که به نیش و و 
هر شیر شرزه را که به نیش سنان گزید

هرکه از حق به و و و 
هرکه از حق به سوی او نظریست

گفت این از و و و 
گفت این از خدای باید خواست

کلاه لاله که لعل است و و و 
کلاه لاله که لعل است اگر تو بشناسی


## Bigram

In [ ]:
unigram = create_unigram(delta=0.96)
bigram = create_bigram(delta=0.88, unigram=unigram)

### Perplexity

In [ ]:
for text in ["train", "test", "valid"]:
  path = f"/content/content/HW1-datasets/{text}.txt"
  print(f"Bigram perplexity of {text} = {bigram_perplexity(path)}")

Bigram perplexity of train = 322.6322649838004
Bigram perplexity of test = 1326.7163336443393
Bigram perplexity of valid = 1318.4189504258493


### Quantitative Test

In [ ]:
for dataset in ["train", "valid", "test"]:

  X_test, Y_test = create_statistical_dataset(path=f"/content/content/HW1-datasets/{dataset}.txt",
                                              number_words=1,
                                              data_number_limit=10*1000)  

  Y_test_predict = []

  for i in range(len(Y_test)):
    if i%500 == 0:
      print(f"i = {i}")

    Y_test[i]
    best_word = None
    best_probability = -math.inf
    last_word = X_test[i][0]

    for condidate_word in vocabulary:
      if bigram(last_word, condidate_word) > best_probability:
          best_probability = bigram(last_word, condidate_word)
          best_word = condidate_word

    Y_test_predict.append(best_word)

  print(f"{dataset} Accuracy = {accuracy_score(Y_test, Y_test_predict)}")

Maximum Data = 912870
i = 0
i = 500
i = 1000
i = 1500
i = 2000
i = 2500
i = 3000
i = 3500
i = 4000
i = 4500
i = 5000
i = 5500
i = 6000
i = 6500
i = 7000
i = 7500
i = 8000
i = 8500
i = 9000
i = 9500
train Accuracy = 0.1208
Maximum Data = 114079
i = 0
i = 500
i = 1000
i = 1500
i = 2000
i = 2500
i = 3000
i = 3500
i = 4000
i = 4500
i = 5000
i = 5500
i = 6000
i = 6500
i = 7000
i = 7500
i = 8000
i = 8500
i = 9000
i = 9500
valid Accuracy = 0.0768
Maximum Data = 73199
i = 0
i = 500
i = 1000
i = 1500
i = 2000
i = 2500
i = 3000
i = 3500
i = 4000
i = 4500
i = 5000
i = 5500
i = 6000
i = 6500
i = 7000
i = 7500
i = 8000
i = 8500
i = 9000
i = 9500
test Accuracy = 0.0799


### Qualitative Test

In [ ]:
file = open("/content/content/HW1-datasets/test_incomplete.txt")
incomplete_lines = file.readlines()

file = open("/content/content/HW1-datasets/test_incomplete_gold.txt")
incomplete_gold_lines = file.readlines()

for i in range(len(incomplete_lines)):
    line = incomplete_lines[i]
    digit = int(line[0])
    line = line[4:]
    line = line.strip()
    words = line.split()
    
    last_word = words[-1]
    
    print(line, end=" ")
    
    for _ in range(digit):        
        best_word = None
        best_probability = -math.inf
        
        for condidate_word in vocabulary:
            if bigram(last_word, condidate_word) > best_probability:
                best_probability = bigram(last_word, condidate_word)
                best_word = condidate_word
            
        print(f"\x1b[32m{best_word}\x1b[0m", end= " ")
        last_word = best_word
    
    print()
    
    print(incomplete_gold_lines[i])  

این سخن حقست اگر نزد سخن گستر و 
این سخن حقست اگر نزد سخن گستر برند

آنکه با یوسف صدیق چنین خواهد کرد 
آنکه با یوسف صدیق چنین خواهد کرد

هیچ دانی چکند صحبت او با تو 
هیچ دانی چکند صحبت او با دگران

سرمه دهی بصر بری سخت خوش است و 
سرمه دهی بصر بری سخت خوش است تاجری

آتش ابراهیم را به دست 
آتش ابراهیم را نبود زیان

من که اندر سر و از 
من که اندر سر جنونی داشتم

هر شیر شرزه را که به نیش و از 
هر شیر شرزه را که به نیش سنان گزید

هرکه از حق به دست و از 
هرکه از حق به سوی او نظریست

گفت این از آن که در 
گفت این از خدای باید خواست

کلاه لاله که لعل است و از آن 
کلاه لاله که لعل است اگر تو بشناسی


# Neural Language Model

## Vocabulary

In [ ]:
file = open("/content/content/HW1-datasets/train.txt")

vocabulary = set()

for i, line in enumerate(file.readlines()):
  if i < 30*1000:
    words = line.split()

    for word in words:
      vocabulary.add(word)

vocabulary = sorted(vocabulary)
print(f"Vocabulary Size = {len(vocabulary)}")

Vocabulary Size = 21188


## Model

In [ ]:
from bisect import bisect_left

class NLM(keras.Model):
  def __init__(self, vocabulary_size, input_size, embedding_size=128, hidden_size=256):
    super().__init__()
    self.input_size = input_size
    self.model = keras.models.Sequential([
                                          keras.layers.Input((input_size,), name="Input"),
                                          keras.layers.Embedding(vocabulary_size + 1, embedding_size, name="Embedding"),
                                          keras.layers.Flatten(name="Flatten"),
                                          keras.layers.Dense(units=hidden_size, name="Hidden"),
                                          keras.layers.Dense(units=vocabulary_size + 1, activation="softmax", name="Output"),
    ])

  def call(self, inputs):
    return self.model(inputs)

  def single_predict(self, words):
    indexes = []
    for word in words:
      index = convert_word_to_index(word)
      indexes.append(index)
    
    indexes = tf.constant(indexes)
    indexes = tf.expand_dims(indexes, 0)

    model_output =  self.model(indexes)

    predicted_word_indexes = keras.backend.argmax(model_output, axis=1)
    predicted_word_index = tf.squeeze(predicted_word_indexes) 
    predicted_word = convert_index_to_word(predicted_word_index)
    return predicted_word

  def perplexity(self, path):

    file = open(path)

    counter = 0
    probability = 0

    for line_number, line in enumerate(file.readlines()):
      words = line.split()

      for i in range(self.input_size, len(words)):
        target_word = words[i]
        target_index = convert_word_to_index(target_word)

        indexes = []
        for j in range(self.input_size):
          index = convert_word_to_index(words[i-j-1])
          indexes.append(index)

        indexes = tf.constant(indexes)
        indexes = tf.expand_dims(indexes, 0)

        model_output =  self.model(indexes)
        model_output = model_output.numpy()[0]

        p_target = model_output[target_index]

        probability += math.log(p_target)
        counter += 1

    log_perplexity = -1/counter*probability  

    perplexity = math.exp(log_perplexity)

    return perplexity

## Functions

In [ ]:
def convert_word_to_index(word):
  pos = bisect_left(vocabulary, word, 0, len(vocabulary))
  return pos if pos != len(vocabulary) and vocabulary[pos] == word else len(vocabulary)

In [ ]:
def convert_index_to_word(index):
  if index==len(vocabulary):
    return "Unknown"
  else:
    return vocabulary[index]

In [ ]:
def create_neural_dataset(path, number_words, line_limit=None, data_number_limit=None):
  file = open(path)

  X_dataset = []
  Y_dataset = []

  for i, line in enumerate(file.readlines()):
    if line_limit is not None and i >= line_limit:
      break

    words = line.split()

    for j in range(number_words, len(words)):
      indexes = []
      for k in range(number_words):
        word = words[j-k-1]
        index = convert_word_to_index(word) 
        indexes.append(index)
      
      X_dataset.append(indexes)

      word_target = words[j]
      index_target = convert_word_to_index(word_target)
      Y_dataset.append(index_target)

  print(f"Maximum Data = {len(X_dataset)}")
  
  if data_number_limit is not None:
    random.seed(0)
    X_dataset, Y_dataset = zip(*random.sample(list(zip(X_dataset, Y_dataset)), data_number_limit))

  X_dataset = np.array(X_dataset)
  Y_dataset = tf.one_hot(indices=Y_dataset, depth=len(vocabulary)+1).numpy()
  return X_dataset, Y_dataset

## Bigram

### Craete and Train

In [ ]:
bigram_nlm = NLM(vocabulary_size=len(vocabulary), input_size=1)

In [ ]:
bigram_nlm.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
X_train, Y_train = create_neural_dataset(path="/content/content/HW1-datasets/train.txt",
                                         number_words=1,
                                         line_limit=30*1000,
                                         data_number_limit=50*1000)

Maximum Data = 181278


In [ ]:
X_valid, Y_valid = create_neural_dataset(path="/content/content/HW1-datasets/valid.txt",
                                         number_words=1,
                                         data_number_limit=25*1000)

Maximum Data = 114079


In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor="val_accuracy", restore_best_weights=True, patience=2)

In [ ]:
bigram_nlm.fit(x=X_train, y=Y_train, validation_data= (X_valid, Y_valid), epochs=10, callbacks=[es_callback])

Epoch 1/10
1563/1563 [==============================] - 197s 125ms/step - loss: 7.7993 - accuracy: 0.0503 - val_loss: 7.5978 - val_accuracy: 0.0543
Epoch 2/10
1563/1563 [==============================] - 180s 115ms/step - loss: 6.9813 - accuracy: 0.0643 - val_loss: 7.6538 - val_accuracy: 0.0633
Epoch 3/10
1563/1563 [==============================] - 176s 112ms/step - loss: 6.4219 - accuracy: 0.0870 - val_loss: 7.9092 - val_accuracy: 0.0624
Epoch 4/10
1563/1563 [==============================] - 178s 114ms/step - loss: 5.7498 - accuracy: 0.1203 - val_loss: 8.2626 - val_accuracy: 0.0553


### Perplexity

In [ ]:
bigram_nlm.perplexity(path="/content/content/HW1-datasets/test.txt")

2100.5970730407657

In [ ]:
bigram_nlm.perplexity(path="/content/content/HW1-datasets/train.txt")

1970.5638742960314

In [ ]:
bigram_nlm.perplexity(path="/content/content/HW1-datasets/valid.txt")

2113.1882825600264

### Quantitative Test

In [ ]:
print(f"Train Accuracy = {bigram_nlm.evaluate(x=X_train, y=Y_train, verbose=1)[1]}")
print(f"Valid Accuracy = {bigram_nlm.evaluate(x=X_valid, y=Y_valid, verbose=1)[1]}")

1563/1563 [==============================] - 45s 29ms/step - loss: 6.4162 - accuracy: 0.0835
Train Accuracy = 0.08349999785423279
782/782 [==============================] - 23s 29ms/step - loss: 7.6538 - accuracy: 0.0633
Valid Accuracy = 0.06328000128269196


In [ ]:
del X_train, Y_train, X_valid, Y_valid

In [ ]:
X_test, Y_test = create_neural_dataset(path="/content/content/HW1-datasets/test.txt",
                                       number_words=1,
                                       data_number_limit=50*1000)

Maximum Data = 73199


In [ ]:
print(f"Test Accuracy = {bigram_nlm.evaluate(x=X_test, y=Y_test, verbose=1)[1]}")

1563/1563 [==============================] - 62s 40ms/step - loss: 7.6502 - accuracy: 0.0619
Test Accuracy = 0.06193999946117401


In [ ]:
del X_test, Y_test

### Qualitative Test

In [ ]:
file = open("/content/content/HW1-datasets/test_incomplete.txt")
incomplete_lines = file.readlines()

file = open("/content/content/HW1-datasets/test_incomplete_gold.txt")
incomplete_gold_lines = file.readlines()

for i in range(len(incomplete_lines)):
    line = incomplete_lines[i]
    digit = int(line[0])
    line = line[4:]
    line = line.strip()
    words = line.split()
    
    last_word = words[-1]

    print(line, end=" ")
    
    for _ in range(digit):
        predicted_word = bigram_nlm.single_predict([last_word])

        print(f"\x1b[32m{predicted_word}\x1b[0m", end= " ")

        last_word = predicted_word
    
    print()
    
    print(incomplete_gold_lines[i])  

این سخن حقست اگر نزد سخن گستر و 
این سخن حقست اگر نزد سخن گستر برند

آنکه با یوسف صدیق چنین خواهد و 
آنکه با یوسف صدیق چنین خواهد کرد

هیچ دانی چکند صحبت او با تو 
هیچ دانی چکند صحبت او با دگران

سرمه دهی بصر بری سخت خوش است و 
سرمه دهی بصر بری سخت خوش است تاجری

آتش ابراهیم را ز من 
آتش ابراهیم را نبود زیان

من که اندر سر و به 
من که اندر سر جنونی داشتم

هر شیر شرزه را که به نیش و به 
هر شیر شرزه را که به نیش سنان گزید

هرکه از حق به جای آن که 
هرکه از حق به سوی او نظریست

گفت این از آن که در 
گفت این از خدای باید خواست

کلاه لاله که لعل است و به جای 
کلاه لاله که لعل است اگر تو بشناسی


## Trigram

### Craete and Train

In [ ]:
trigram_nlm = NLM(vocabulary_size=len(vocabulary), input_size=2)

In [ ]:
trigram_nlm.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
X_train, Y_train = create_neural_dataset(path="/content/content/HW1-datasets/train.txt",
                                         number_words=2,
                                         line_limit=30*1000,
                                         data_number_limit=50*1000)

Maximum Data = 151278


In [ ]:
X_valid, Y_valid = create_neural_dataset(path="/content/content/HW1-datasets/valid.txt",
                                         number_words=2,
                                         data_number_limit=25*1000)

Maximum Data = 95190


In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor="val_accuracy", restore_best_weights=True, patience=2)

In [ ]:
trigram_nlm.fit(x=X_train, y=Y_train, validation_data= (X_valid, Y_valid), epochs=10, callbacks=[es_callback])

Epoch 1/10
1563/1563 [==============================] - 204s 130ms/step - loss: 7.8708 - accuracy: 0.0525 - val_loss: 7.6566 - val_accuracy: 0.0598
Epoch 2/10
1563/1563 [==============================] - 208s 133ms/step - loss: 7.0173 - accuracy: 0.0678 - val_loss: 7.7522 - val_accuracy: 0.0665
Epoch 3/10
1563/1563 [==============================] - 191s 122ms/step - loss: 6.2917 - accuracy: 0.0914 - val_loss: 8.0673 - val_accuracy: 0.0626
Epoch 4/10
1563/1563 [==============================] - 183s 117ms/step - loss: 5.2346 - accuracy: 0.1501 - val_loss: 8.6263 - val_accuracy: 0.0522


### Perplexity

In [ ]:
trigram_nlm.perplexity(path="/content/content/HW1-datasets/test.txt")

2289.0963513577467

In [ ]:
trigram_nlm.perplexity(path="/content/content/HW1-datasets/train.txt")

2113.4442356396216

In [ ]:
trigram_nlm.perplexity(path="/content/content/HW1-datasets/valid.txt")

2309.0123362550376

### Quantitative Test

In [ ]:
trigram_nlm.evaluate(x=X_train, y=Y_train, verbose=1)
trigram_nlm.evaluate(x=X_valid, y=Y_valid, verbose=1)

782/782 [==============================] - 21s 27ms/step - loss: 7.7522 - accuracy: 0.0665


[7.752236843109131, 0.06648000329732895]

In [ ]:
del X_train, Y_train, X_valid, Y_valid

In [ ]:
X_test, Y_test = create_neural_dataset(path="/content/content/HW1-datasets/test.txt",
                                       number_words=2,
                                       data_number_limit=50*1000)

Maximum Data = 61074


In [ ]:
trigram_nlm.evaluate(x=X_test, y=Y_test, verbose=1)

1563/1563 [==============================] - 38s 24ms/step - loss: 7.7399 - accuracy: 0.0659


[7.739939212799072, 0.06586000323295593]

In [ ]:
del X_test, Y_test

### Qualitative Test

In [ ]:
file = open("/content/content/HW1-datasets/test_incomplete.txt")
incomplete_lines = file.readlines()

file = open("/content/content/HW1-datasets/test_incomplete_gold.txt")
incomplete_gold_lines = file.readlines()

for i in range(len(incomplete_lines)):
    line = incomplete_lines[i]
    digit = int(line[0])
    line = line[4:]
    line = line.strip()
    words = line.split()
    
    
    last_word = words[-1]
    next_last_word = words[-2]

    print(line, end=" ")
    
    for _ in range(digit):
        predicted_word = trigram_nlm.single_predict([next_last_word, last_word])

        print(f"\x1b[32m{predicted_word}\x1b[0m", end= " ")

        next_last_word = last_word
        last_word = predicted_word
  
    print()
    
    print(incomplete_gold_lines[i])

این سخن حقست اگر نزد سخن گستر به 
این سخن حقست اگر نزد سخن گستر برند

آنکه با یوسف صدیق چنین خواهد به 
آنکه با یوسف صدیق چنین خواهد کرد

هیچ دانی چکند صحبت او با را 
هیچ دانی چکند صحبت او با دگران

سرمه دهی بصر بری سخت خوش است و 
سرمه دهی بصر بری سخت خوش است تاجری

آتش ابراهیم را و به 
آتش ابراهیم را نبود زیان

من که اندر سر تو و 
من که اندر سر جنونی داشتم

هر شیر شرزه را که به نیش تو و 
هر شیر شرزه را که به نیش سنان گزید

هرکه از حق به و تو دل 
هرکه از حق به سوی او نظریست

گفت این از دل آن که 
گفت این از خدای باید خواست

کلاه لاله که لعل است و و به 
کلاه لاله که لعل است اگر تو بشناسی
این سخن حقست اگر نزد سخن گستر به 
این سخن حقست اگر نزد سخن گستر برند

آنکه با یوسف صدیق چنین خواهد به 
آنکه با یوسف صدیق چنین خواهد کرد

هیچ دانی چکند صحبت او با را 
هیچ دانی چکند صحبت او با دگران

سرمه دهی بصر بری سخت خوش است و 
سرمه دهی بصر بری سخت خوش است تاجری

آتش ابراهیم را و به 
آتش ابراهیم را نبود زیان

من که اندر سر تو و 
من که اندر سر جنونی داشتم

هر شیر شرزه را که به نیش تو و 
هر شیر شرزه را که